In [4]:
import numpy as np
import sympy as sym
from sympy import Function, hessian, pprint, Matrix, simplify
import math
from scipy.spatial.transform import Rotation as R

d, a,q,alpha = sym.symbols('d a q alpha')

def print_pose(pose):
    print("position of robot flange:", pose[0:3,3])
    print ("x vector:" , pose[0:3:,0])
    print ("y vector:" , pose[0:3:,1])
    print ("z vector:" , pose[0:3:,2])    

# symbolic DH matrix
def get_symbolic_dh_matrix (d,a,q,alpha):
    dh = sym.eye(4)
    dh[0,0] = sym.cos(q)
    dh[0,1] = -sym.sin(q) * sym.cos(alpha)
    dh[0,2] = sym.sin(q) * sym.sin(alpha)
    dh[0,3] = a * sym.cos(q)
    dh[1,0] = sym.sin(q)
    dh[1,1] = sym.cos(q)*sym.cos(alpha)
    dh[1,2] = -sym.cos(q)* sym.sin(alpha)
    dh[1,3] = a *sym.sin(q)
    dh[2,1] = sym.sin(alpha)
    dh[2,2] = sym.cos(alpha)
    dh[2,3] = d

    return dh

dh1 = get_symbolic_dh_matrix(d,a,q,alpha)
f = sym.lambdify([d, a,q,alpha], dh1, "numpy") 

# calculate DH matrix with given parameters and angle q
def get_matrix_for_angle (d,a,q0,alpha, q):
    matrix = f(d,a,np.deg2rad(q0 +q),np.deg2rad(alpha))
    return matrix
    
def get_pose (q,kinematics):
    resulting_transform = np.eye(4)
    for i in range(6):
        resulting_transform = resulting_transform @ get_matrix_for_angle(*kinematics[i], q[i])
    return resulting_transform

# 24 nominal parameters of kinematics
def get_nominal_DH_kinematic_parameters():
    kinematic_parameters =                                  np.array([525,150,0,-90], dtype = 'float')
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([0,790,-90,180])))
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([0,250,0,-90])))    
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([-1040,0,0,90])))
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([0,0,0,-90])))
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([-100,0,0,0])))
    return kinematic_parameters

def get_kinematics_model_with_noise(kinematic_parameters):
    for param in np.nditer(kinematic_parameters, op_flags=['readwrite']):
        param[...] = param + np.random.normal(0,0.15)
    return kinematic_parameters

# vector q in deg
q = np.array ([0,20,0,0,0,0])

nomibal_kinematics = get_nominal_DH_kinematic_parameters()
print("nominal kinematics:\n", nomibal_kinematics)
nominal_pose = get_pose(q,nomibal_kinematics)
print_pose(nominal_pose)

real_kinematics = get_kinematics_model_with_noise(nomibal_kinematics.copy())
print("kinematics with noise:\n", real_kinematics)
real_pose = get_pose(q,real_kinematics)
print_pose(real_pose)



nominal kinematics:
 [[  525.   150.     0.   -90.]
 [    0.   790.   -90.   180.]
 [    0.   250.     0.   -90.]
 [-1040.     0.     0.    90.]
 [    0.     0.     0.   -90.]
 [ -100.     0.     0.     0.]]
position of robot flange: [ 1.57695054e+03 -1.05771358e-13  1.11237736e+03]
x vector: [ 3.42020143e-01 -5.75395780e-17  9.39692621e-01]
y vector: [5.75395780e-17 1.00000000e+00 4.02896463e-17]
z vector: [-9.39692621e-01  4.02896463e-17  3.42020143e-01]
kinematics with noise:
 [[ 5.25082456e+02  1.49902433e+02  1.30273388e-01 -8.98493932e+01]
 [ 3.22061606e-01  7.90144080e+02 -8.99394121e+01  1.79923166e+02]
 [ 1.66601724e-01  2.50128426e+02 -2.11034304e-01 -8.97992720e+01]
 [-1.04017579e+03  1.08478158e-01 -1.17545035e-02  8.98773004e+01]
 [ 1.94767271e-01 -6.12963601e-02  9.52888379e-02 -9.01722621e+01]
 [-1.00194703e+02  1.05266411e-01 -2.29560424e-01 -3.93595501e-02]]
position of robot flange: [1577.43403958    3.965732   1107.12240753]
x vector: [ 0.34491094 -0.00589973  0.9386

In [208]:
import numpy as np
import sympy as sym
from sympy import Function, hessian, pprint, Matrix, simplify

d, a,q,alpha = sym.symbols('d a q alpha')

def get_dh (d,a,q,alpha):
    dh = sym.eye(4)
    dh[0,0] = sym.cos(q)
    dh[0,1] = -sym.sin(q) * sym.cos(alpha)
    dh[0,2] = sym.sin(q) * sym.sin(alpha)
    dh[0,3] = a * sym.cos(q)
    dh[1,0] = sym.sin(q)
    dh[1,1] = sym.cos(q)*sym.cos(alpha)
    dh[1,2] = -sym.cos(q)* sym.sin(alpha)
    dh[1,3] = a * sym.sin(q)
    dh[2,1] = sym.sin(alpha)
    dh[2,2] = sym.cos(alpha)
    dh[2,3] = d
    print(dh)
    return dh

get_dh (d,a,q,alpha)

Matrix([[cos(q), -sin(q)*cos(alpha), sin(alpha)*sin(q), a*cos(q)], [sin(q), cos(alpha)*cos(q), -sin(alpha)*cos(q), a*sin(q)], [0, sin(alpha), cos(alpha), d], [0, 0, 0, 1]])


Matrix([
[cos(q), -sin(q)*cos(alpha),  sin(alpha)*sin(q), a*cos(q)],
[sin(q),  cos(alpha)*cos(q), -sin(alpha)*cos(q), a*sin(q)],
[     0,         sin(alpha),         cos(alpha),        d],
[     0,                  0,                  0,        1]])